<a href="https://colab.research.google.com/github/rhudaina/Linear-Systems-and-Applications-A-Hands-On-Python-Workshop/blob/main/Day2/Day2_Lecture_2_IterativeMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sp

# Sparse Matrices

A $n\times m$ matrix is **sparse** if it has few non-zero entries in comparison to all $nm$ total entries.  Sparsity is a qualitative notion - it might mean we have $O(\min\{n,m\})$ non-zero entries (for example, a diagonal matrix), it might also mean we have $O(nm)$ entries, but the constant is small (for example, $mn/100$).  A **dense** matrix is not sparse, meaning that most (or all) of the entries are non-zero.

Recall the formula for matrix-vector multiplication:
$$y_i = \sum_j A_{i,j} x_j$$

When we multiply a vector (or matrix) by a sparse matrix $A$, most of the coefficients are zero, and so we might expect that we can apply the matrix more quickly than we might apply a dense matrix.  We can rewrite the matrix-vector multiplication formula as
$$y_i = \sum_{j\in nz(i)} A_{i,j} x_j$$

Where $nz(i)$ denotes the column indices $j$ for which $A_{i,j}$ is non-zero.  We see the complexity of multiplying a sparse matrix is $O(nnz(A))$, where $nnz(A)$ is the number of non-zeros (note that when $A$ is dense, $nnz(A) = mn$).

## Sparse Matrix Formats

There are a variety of ways sparse matrices are stored in practice.  The utility of each format depends on whether there is any structure in the non-zeros, or what the matrix will be used for.

Scipy provides several standard types of sparse matrices in `sicpy.sparse`.  See the [documentation](https://docs.scipy.org/doc/scipy/reference/sparse.html#sparse-matrix-classes).


### COOrdinate Format

Perhaps the easiest to describe is the COO (COOrdinate format), which just stores three lists `i,j,data`, where `i[k]` and  `j[k]` are the row and column indices for a non-zero entry with value `data[k]`.

Scipy documentation is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix)

In [ ]:
row  = [0,2,1]
col  = [0,2,1]
val  = [1,1,1]

S = sp.sparse.coo_matrix((val, (row,col)), shape=(3,3))
print(S)

Sparse matrix formats have a `toarray` command which converts to a dense array.

In [ ]:
S.toarray()

Another example:

In [ ]:
row  = [0,1,2,2]
col  = [0,1,2,0]
val  = [1,1,1,2]

S = sp.sparse.coo_matrix((val, (row,col)), shape=(3,3))
S.toarray()

You can visualize the sparsity pattern using PyPlot's `spy` function (this is particularly useful for large sparse matrices)

In [ ]:
import matplotlib.pyplot as plt
plt.spy(S)
plt.show()

### Compressed Sparse Row/Column Formats

One of the disadvantages of COO Matrices are that entries need not be ordered in any way, which can lead to inefficiencies in memory access in matrix-vector or matrix-matrix multiplication.

Commonly used formats which keeps entries in a sensible order (without additional structure assumed) are Compressed Sparse Row (CSR) and Compressed Sparse Column (CSC) matrices.  You might think of these as the sparse equivalents of row-major and column-major dense matrices.

We'll describe CSC matrices - the transpose is a CSR matrix.

If `S` is a CSC matrix with `m` rows, `n` columns, and `nnz` non-zeros, we specify `S` with three lists: `ptr` (length `n+1`), `row` (length `nnz`) and `val` (length `nnz`).  The non-zero row indices for column `j` are stored in `row[ptr[j]:ptr[j+1]]`, and the non-zero values for those rows are stored in `val[ptr[j]:ptr[j+1]]`.  If you're familiar with pointers in a language like C, `ptr` is an array of pointer offsets.

Basically, the non-zero entries for each column are stored in contiguous blocks of memory.  This can make it much faster to apply CSC matrices.

Scipy documentation is [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html#scipy.sparse.csc_matrix)

In [ ]:
ptr = [0,2,4,5]
row = [0,2,0,1,2]
val = [1,2,3,1,1]

S = sp.sparse.csc_matrix((val, row, ptr), shape=(3,3))
S.toarray()

You can use `toarray` command to get a numpy array without the matrix wrapper

In [ ]:
# the pointer list gives you slices to get the data for each column
j = 1
row[ptr[j]:ptr[j+1]], val[ptr[j]:ptr[j+1]]

### Other Sparse Matrix Types

Other matrix types in `scipy.sparse` include:
* [`dia_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dia_matrix.html#scipy.sparse.dia_matrix), which is good for diagonal/banded matrices
* [`lil_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html#scipy.sparse.lil_matrix), or a (row-based) list-of-lists matrix, which is good for mutating row operations
* [`bsr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.bsr_matrix.html#scipy.sparse.bsr_matrix), or block sparse row, which is good for sparse matrices with dense blocks
* [`dok_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html#scipy.sparse.dok_matrix), or dictionary of keys, which is good for when you want to access and change individual entries quickly.

In [ ]:
data = np.array([[1,2,3],[4,5,6]])
S = sp.sparse.dia_matrix(
  (data, [0,1]),
  shape=(3,3))
S.toarray()

In [ ]:
A = np.eye(5) # identity
As = sp.sparse.dia_matrix(A)
As

To convert between sparse matrix formats, you can use `tocsr`, `tocoo`, etc.

In [ ]:
As1 = As.tocsr()
As1

In [ ]:
As2 = As.tocoo()
As2

## Saving and Loading Sparse Matrices

Dense matrices can be easily stored and read from comma-separated value formats using e.g. `np.loadtxt` and `np.savetxt`.  Because sparse matrices can be stored more efficiently than dense matrices, they have special storage formats.

One source of sparse matrices which is used extensively for testing is the University of Florida Sparse Matrix Collection ([Link](https://sparse.tamu.edu/)).  As an example, we'll just read the `1138_bus.mtx` file, which is matrix-market format, and you can download from that link.  This is a plain text file, with a header (every line begins with `%`), and the first row contains three integers: the number of rows, number of columns, and number of nonzeros in the matrix.  For `1138_bus.mtx`, this looks like:
```
%%MatrixMarket matrix coordinate real symmetric
%-------------------------------------------------------------------------------
% UF Sparse Matrix Collection, Tim Davis
% http://www.cise.ufl.edu/research/sparse/matrices/HB/1138_bus
% name: HB/1138_bus
% [S ADMITTANCE MATRIX 1138 BUS POWER SYSTEM, D.J.TYLAVSKY, JULY 1985.]
% id: 1
% date: 1985
% author: D. Tylavsky
% ed: I. Duff, R. Grimes, J. Lewis
% fields: title A name id date author ed kind
% kind: power network problem
%-------------------------------------------------------------------------------
1138 1138 2596
```
So the matrix is `1138 x 1138` with 2596 nonzeros.
Every subsequent row is in the form `row, column, data` - one nonzero in COO format.

Let's go ahead and load this matrix:

In [ ]:
data = np.loadtxt('1138_bus.mtx', comments='%') # skip any rows that begin with `%`
data.shape

The first non-comment row contains the size of the matrix, so we can handle it separately.

In [ ]:
m, n = int(data[0,0]), int(data[0,1])
data = data[1:]

Matrix market format uses the Fortran convention of beginning indexes at 1, so we need to subtract 1 from indices to get the correct Python indices

In [ ]:
rows = data[:,0] - 1
cols = data[:,1] - 1
vals = data[:,2]
A = sp.sparse.coo_matrix((vals, (rows, cols)), shape=(m,n))

plt.spy(A)
plt.show()

Let's look at the difference between using the sparse matrix and a dense matrix for matrix-vector multiplications:

In [ ]:
import time
x = np.random.randn(n)
y = np.empty_like(x)

t_start = time.time()
y = A @ x
t_end = time.time()
tcoo = t_end - t_start
print("time for COO multiply: %f sec" % tcoo)

In [ ]:
Adense = A.todense()

t_start = time.time()
y = Adense @ x
t_end = time.time()
tdense = t_end - t_start
print("time for dense multiply: %f sec\n" % tdense)

print("COO is %f times faster" % (tdense / tcoo))

Using the sparse matrix is several times faster than using a dense matrix.

# Sparse Linear Algebra

So far, we have seen how sparse matrices and linear operators can be used to speed up basic matrix-vector and matrix-matrix operations, and decrease the memory footprint of the representation of a linear map.

Just as there are special data types for sparse and structured matrices, there are specialized linear algebra routines which allow you to take advantage of sparsity and fast matrix-vector products.

Routines for sparse linear algebra are found in `scipy.sparse.linalg`

In [ ]:
n = 100
A = sp.sparse.random(n, n, 0.01) + sp.sparse.eye(n)
A

## Sparse Direct Methods

In [ ]:
plt.spy(A)
plt.show()

This typically refers to producing a factorization of a sparse matrix for use in solving linear systems.

The thing to keep in mind is that many factorizations will generally be dense, even if the original matrix is sparse, e.g. eigenvalue decompositions, QR decomposition, SVD, etc.  This means that if we compute a factorization, we are going to lose all the advantages we had from sparsity.  

What we really want is a factorization where if `A` is sparse, the terms in the factorization are also sparse.  The factorization where this is easiest to achieve is the LU decomposition.  In general, the `L` and `U` terms will be more dense than `A`, and sometimes much more dense.  However, we can seek a permuted version of the matrix `A` which will minimize the amount of "fill-in" which occurs.  This is often done using "nested disection" algorithm, which is outside the scope of this course.  If you ever need to do this explicitly, the [METIS package](http://glaros.dtc.umn.edu/gkhome/metis/metis/overview) is commonly used.

We'll just use the function [`scipy.linalg.splu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html#scipy.sparse.linalg.splu) (SParse LU) at a high level, which produces a factorization object that can be used to solve linear systems.

In [ ]:
A = A.tocsc() # need to convert to CSC form first
LU = sp.sparse.linalg.splu(A)
LU

The resulting object stores the factors necessary to compute `A = PLUQ` (`P` permutes rows, and `Q` permutes columns).  It is computed using the [SuperLU library](https://portal.nersc.gov/project/sparse/superlu/).  Typically, you will just use the `solve` method on this object.

In [ ]:
x = np.random.randn(n)
y = A @ x

x2 = LU.solve(y)
print(np.linalg.norm(x2 - x))

You can also use the `scipy.sparse.linalg.spsolve` function, which wraps this factorization.

In [ ]:
x2 = sp.sparse.linalg.spsolve(A, y)
print(np.linalg.norm(x2 - x))

# Iterative Methods

Generate a large sparse random matrix $A$.

In [ ]:
n = 1000
A = sp.sparse.random(n, n, density=0.0001, format='csr') + sp.sparse.eye(n, format='csr') * n

plt.spy(A)
plt.show()

In [ ]:
x = np.random.rand(n)
y = A @ x

## Jacobi Iteration

In [ ]:
def solve_jacobi(A, y, x0, TOL, maxIter):
  for k in range(maxIter):
    x = np.zeros(n)
    for i in range(n):
      x[i] = y[i] - sum(A[i,0:i]*x0[0:i]) - sum(A[i,i+1:n]*x0[i+1:n])
      x[i] = x[i]/A[i,i]

    if max(abs(x-x0))<TOL:
      break

    x0 = x

  if k == maxIter-1:
    print('\nMaximum number of iterations reached!')

  return x

In [ ]:
x0 = np.ones(n)   # initial guess
TOL = 1.0e-10     # tolerance
maxIter = 1000    # maximum number of iterations
Adense = A.toarray()

t_start = time.time()
x1 = solve_jacobi(Adense, y, x0, TOL, maxIter)
t_end = time.time()

print("elapsed time:", t_end, "sec")
print(np.linalg.norm(x1 - x))

## Sparse Iterative Methods

Sparse iterative methods are another class of methods you can use for solving linear systems built on [Krylov subspaces](https://en.wikipedia.org/wiki/Krylov_subspace).  They only require matrix-vector products, and are ideally used with sparse matrices and fast linear operators.  You can typically learn the theory behind these methods in a numerical linear algebra course - we'll just talk about how to use them.

All these methods are meant to solve linear systems: find `x` so that `A @ x = b`, or least squares problems minimizing `norm(A @ x - b)`

You can find a list of options in the [documentation for `scipy.sparse.linalg`](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html#solving-linear-problems).  Here are some common options:

* Conjugate Gradient: `scipy.sparse.linalg.cg` for `A` SPD
* MINRES: `scipy.sparse.linalg.minres` for `A` symmetric
* GMRES: `scipy.sparse.linalg.gmres` for general square `A`
* LSQR: `scipy.sparse.linalg.lsqr` for solving least squares problems

For example, we can use `gmres` with the same matrix we used for `splu`:

In [ ]:
x2 = np.empty_like(x)

t_start = time.time()
x2, exit = sp.sparse.linalg.gmres(A, y)
t_end = time.time()
print("GMRES:", t_end - t_start, "sec")
print(np.linalg.norm(x2 - x))

In [ ]:
t_start = time.time()
x3 = sp.sparse.linalg.spsolve(A, y)
t_end = time.time()
print("spsolve:", t_end - t_start, "sec")
print(np.linalg.norm(x3 - x))

# Reference

1.   [Brad Nelson (2021), Scientific Computing with Python](https://caam37830.github.io/book/index.html)

